In [1]:
#import findspark
#findspark.init()
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as func


spark = SparkSession.builder \
    .master("local[2]") \
    .appName("COM6012 Assignment 1 Task1") \
    .getOrCreate()

sc = spark.sparkContext


In [2]:
report_list = []

In [3]:
logFile=spark.read.text("NASA_access_log_Jul95.gz").cache()

In [4]:
logFile.count()

1891715

In [5]:
reqs = logFile.withColumn('time', func.regexp_extract(logFile['value'],':(.*) -',1))

In [6]:
reqs_with_data = reqs.withColumn('Date', func.regexp_extract(func.col('value'),'\[(\d+)\/Jul',1))

In [7]:
df_first_slot = reqs_with_data.filter(func.col("time").between("00:00:00","03:59:59"))

In [8]:
avg_1=df_first_slot.groupBy('Date').count().select(func.mean('count').alias("Average number of request on 0 to 4 clock")).collect()[0]

In [9]:
df_second_slot = reqs_with_data.filter(func.col("time").between("04:00:00","07:59:59"))

In [10]:
avg_2=df_second_slot.groupBy('Date').count().select(func.mean('count').alias("Average number of request on 4 to 8 clock")).collect()[0]

In [11]:
df_third_slot = reqs_with_data.filter(func.col("time").between("08:00:00","11:59:59"))

In [12]:
avg_3=df_third_slot.groupBy('Date').count().select(func.mean('count').alias("Average number of request on 8 to 12 clock")).collect()[0]

In [13]:
df_fourth_slot = reqs_with_data.filter(func.col("time").between("12:00:00","15:59:59"))

In [14]:
avg_4=df_fourth_slot.groupBy('Date').count().select(func.mean('count').alias("Average number of request on 12 to 16 clock")).collect()[0]

In [15]:
df_fifth_slot = reqs_with_data.filter(func.col("time").between("16:00:00","19:59:59"))

In [16]:
avg_5=df_fifth_slot.groupBy('Date').count().select(func.mean('count').alias("Average number of request on 16 to 20 clock")).collect()[0]

In [17]:
df_sixth_slot = reqs_with_data.filter(func.col("time").between("20:00:00","23:59:59"))

In [18]:
avg_6=df_fifth_slot.groupBy('Date').count().select(func.mean('count').alias("Average number of request on 20 to 24 clock")).collect()[0]

In [19]:
rdd_report = sc.parallelize([('From 0 to 4',avg_1),
                             ('From 4 to 8',avg_2),
                             ('From 8 to 12',avg_3),
                             ('From 12 to 16',avg_4),
                             ('From 16 to 20',avg_5),
                             ('From 20 to 24',avg_6)])
df_report = rdd_report.toDF([" Six different Timeslot","Average number of request of Jul"])

In [28]:
df_report.show()

+-----------------------+--------------------------------+
| Six different Timeslot|Average number of request of Jul|
+-----------------------+--------------------------------+
|            From 0 to 4|             [7079.714285714285]|
|            From 4 to 8|             [5479.392857142857]|
|           From 8 to 12|             [14462.32142857143]|
|          From 12 to 16|            [17377.571428571428]|
|          From 16 to 20|            [13580.333333333334]|
|          From 20 to 24|            [13580.333333333334]|
+-----------------------+--------------------------------+



In [44]:
reqs_file = logFile.withColumn('File_name', func.regexp_extract(logFile['value'],'\"GET (.*)HTTP',1))

In [45]:
df_new = reqs_file.filter(reqs_file['File_name'].contains(".html"))

In [46]:
reqs_df_count=df_new.groupBy('File_name').count().sort('count', ascending=False)

In [47]:
reqs_df_count.show(20,False)

+------------------------------------------------+-----+
|File_name                                       |count|
+------------------------------------------------+-----+
|/ksc.html                                       |39854|
|/shuttle/missions/missions.html                 |24745|
|/shuttle/countdown/liftoff.html                 |21946|
|/shuttle/missions/sts-71/mission-sts-71.html    |16619|
|/shuttle/missions/sts-70/mission-sts-70.html    |16056|
|/shuttle/missions/sts-71/images/images.html     |15861|
|/history/apollo/apollo.html                     |14437|
|/history/apollo/apollo-13/apollo-13.html        |13742|
|/history/history.html                           |11675|
|/shuttle/countdown/countdown.html               |8548 |
|/shuttle/technology/sts-newsref/stsref-toc.html |7363 |
|/shuttle/missions/sts-69/mission-sts-69.html    |6926 |
|/software/winvn/winvn.html                      |6768 |
|/shuttle/missions/sts-70/images/images.html     |6702 |
|/shuttle/missions/sts-71/movie